In [2]:
%load_ext autoreload
%autoreload 2

In [12]:
# cd workspace/theislab/mubind/notebooks/sparse

In [4]:
import mubind as mb
import numpy as np
import pandas as pd
import torch
import bindome as bd
bd.constants.ANNOTATIONS_DIRECTORY = '../../annotations'
# mb.models.Mubind
import torch.optim as topti
import torch.utils.data as tdata
import matplotlib.pyplot as plt
import logomaker

# Use a GPU if available, as it should be faster.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device: " + str(device))

/home/rio/miniconda3/envs/mubind/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda:0


In [5]:
import itertools

def _mismatch(word, letters, num_mismatches):
    for locs in itertools.combinations(range(len(word)), num_mismatches):
        this_word = [[char] for char in word]
        for loc in locs:
            orig_char = word[loc]
            this_word[loc] = [l for l in letters if l != orig_char]
        for poss in itertools.product(*this_word):
            yield "".join(poss)



In [6]:
# motif = "ATT"
# x2, y2 = mb.datasets.simulate_xy(motif, n_trials=2100, seqlen=len(motif) + 1, max_mismatches=min(len(motif), 2), counts_size=1)
# x2, y2
# df = pd.DataFrame(data=y2)
# df.columns = [1]
# df.index = x2
# df[0] = 0
# df = df[[0, 1]]

In [7]:
df = mb.bindome.datasets.ProBound.ctcf(flank_length=0)
df = df.sort_values(1, ascending=False).reset_index(drop=True)
# data = df.head(1000)
# data = df.copy()
data = df.sample(1500, random_state=50)
# data = df.copy()
data.index = data['seq']

# remove as many nucleotides as posisble (faster training/convergence)
data.index = data.index.astype(str).str[45:]


# data.index = data.index.astype(str).str[-15:]
del data['seq']

In [8]:
data.shape

(1500, 2)

In [9]:
data = data.sample(data.shape[0], random_state=100)

In [10]:


from matplotlib import rcParams
rcParams['figure.figsize'] = 5, 1

In [27]:
# TODO: skipped for now
n_rounds = 1

print('loading object (# entries)', data.shape[0])
dataset = mb.datasets.SelexDataset(data, n_rounds=n_rounds, labels=[0, 1])
train = tdata.DataLoader(dataset=dataset,
                         # batch_size=256,
                         batch_size=512,
                         shuffle=False)


loading object (# entries) 1500

